In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wikipedia

In [2]:
movie_items =  "movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western"
items_columns = movie_items.split(' | ')                               ## For u.item file in movielens-100k dataset
items = pd.read_csv('ml-100k/u.item',sep='|',names=items_columns,encoding='latin-1')

In [3]:
print(items.shape)
items.head()

(1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
genres = list(items.columns[-18:])
genre_list = []
for i in range(len(items)):
    str1 = ','.join(list(items[genres].loc[i][items[genres].loc[i]==1].index))
    genre_list.append(str1)
    

In [6]:
items_content_based = items[['movie id','movie title']]

In [7]:
items_content_based['genres'] = genre_list

<ipython-input-7-9a9891128bec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_content_based['genres'] = genre_list


In [8]:
items_content_based

,movie id,movie title,genres
0,1,Toy Story (1995),"Animation,Children's,Comedy"
1,2,GoldenEye (1995),"Action,Adventure,Thriller"
2,3,Four Rooms (1995),Thriller
3,4,Get Shorty (1995),"Action,Comedy,Drama"
4,5,Copycat (1995),"Crime,Drama,Thriller"
...,...,...,...
1677,1678,Mat' i syn (1997),Drama
1678,1679,B. Monkey (1998),"Romance,Thriller"
1679,1680,Sliding Doors (1998),"Drama,Romance"
1680,1681,You So Crazy (1994),Comedy


In [9]:
movie_summary = []
for i in range(len(items_content_based)):
    try:
        wiki_movie = wikipedia.page(wikipedia.search(items['movie title'][i])[0])
        summ = wiki_movie.summary
        movie_summary.append(summ)
    except:
        summ='-'
        movie_summary.append(summ)

In [10]:
items_content_based['movie_summary'] = movie_summary

In [ ]:
# items_content_based[items_content_based['movie_summary']=='-']

In [11]:
def get_more_data(title):
    try:
        movie = wikipedia.page(title)
        summ = wikipedia.summary(title,sentences=10)
    except:
         summ = '-'
    return summ


In [12]:
for i in range(len(items_content_based)):
    if(items_content_based.loc[i]['movie_summary']=='-') or (len(items_content_based.loc[i]['movie_summary'])<=300) :
        items_content_based.loc[i,'movie_summary']= get_more_data(items_content_based.loc[i]['movie title'])

In [13]:
def get_more_data_2(title):
    #try:
    topics = wikipedia.search(title)
    print ("{} may refer to: ".format(title))
    for i, topic in enumerate(topics):
        print (i, topic)
    choice = int(input("Enter a choice: "))
    assert choice in range(len(topics))
    summary = wikipedia.summary(topics[choice],sentences=10)
    #except:
     #   summary = "-"
    
    return summary
    

In [14]:
for i in range(len(items_content_based)):
    if(items_content_based.loc[i,'movie_summary'] =='-'):
        items_content_based.loc[i,'movie_summary'] = get_more_data_2(items_content_based.loc[i,'movie title'])

In [18]:
items_content_based = items_content_based[['movie id','movie title','genres','movie_summary']]
items_content_based.head()

,movie id,movie title,genres,movie_summary
0,1,Toy Story (1995),"Animation,Children's,Comedy",Toy Story is a computer animated Disney media ...
1,2,GoldenEye (1995),"Action,Adventure,Thriller",GoldenEye is a 1995 spy film and the seventeen...
2,3,Four Rooms (1995),Thriller,Four Rooms is a 1995 American anthology comedy...
3,4,Get Shorty (1995),"Action,Comedy,Drama",Get Shorty is a 1995 American gangster crime t...
4,5,Copycat (1995),"Crime,Drama,Thriller",Copycat is a 1995 American psychological thril...


In [19]:
items_content_based.to_csv('items_content_based_final.csv',index=False)